In [1]:
import statistics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import operator

import linear_model_tests as lmt
import decision_tree_opt as dto
%reload_ext autoreload
%autoreload 2

lin_mod = lmt.Test()
opt_tree = dto.Parameters()

C:\Users\matti\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data = pd.read_csv("energydata_complete.csv")
data = data.dropna()
names = ["date","appliance_wh", "light_wh","kitchen_celsius","kitchen_hum_perc",
        "living_celsius","living_hum_perc","laundry_celsius","laundry_hum_perc","office_celsius","office_hum_perc",
         "bathroom_celsius","bathroom_hum_perc","portico_celsius","portico_hum_perc","ironing_celsius","ironing_hum_perc",
         "teen_celsius","teen_hum_perc","parents_celsius","parents_hum_perc","cws_celsius","cws_pressure","cws_hum_perc",
         "cws_wind","cws_visibility","cws_dew_point","rv1","rv2" ]

data = data.rename(columns = dict(zip(data.columns, names)))
data = data.drop(["rv1","rv2","date"], axis = 1)
data = data.drop_duplicates(data.columns, keep = "last")
#data = data.drop(data[(data.appliance_wh > 790)|(data.appliance_wh < 0)].index)
data = data.reset_index()
data = data.drop(["index", "light_wh"], axis = 1)
data.shape

(19735, 25)

In [ ]:
plt.figure(figsize = (18,8))
corr = data.corr()
sns.heatmap(corr, annot = True)

In [22]:
def MSE(true,pred):
    a = round(mean_squared_error(y_true = true, y_pred = pred), 5)
    return a/len(true)

## LINEAR MODEL

In [3]:
X = data.drop("appliance_wh", axis = 1).values
y = data.appliance_wh
X = X.reshape( (len(X), len(X[0])))
y = y.values.reshape( len(y), 1)
print(y.shape, X.shape)

(19735, 1) (19735, 24)


In [4]:
train_size = int(len(X)*0.66)
X_train = X[:train_size]
y_train = y[:train_size]
X_test  = X[train_size:]
y_test  = y[train_size:]

In [26]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

model = LinearRegression(fit_intercept = True)
model.fit(X_train, y_train)

y_pred = model.predict(X)

print ("Train Error:", MSE(y_train, y_pred[:train_size]))
print ("Test Error:",  MSE(y_test, y_pred[train_size:]))
print()
print("R^2 = ", round(r2_score(y, y_pred), 5))

Train Error: 0.74367193243762
Test Error: 1.2450432131147542

R^2 =  0.12153


In [11]:
lasso = lin_mod.lasso(data, X, y)

Reducing features with LASSO 
	 ## Remaining features: 15 
	 ## R^2 after reduction: 0.19733


In [7]:
recursive = lin_mod.recurs_elimin(data, X, y)

Reducing features with Recursive Feature Elimination 
	 ## Remaining features: 14 
	 ## R^2 after reduction: 0.18978


## POLYNOMIAL MODEL

SPLINE: https://www.analyticsvidhya.com/blog/2018/03/introduction-regression-splines-python-codes/

In [27]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree = 2, include_bias = False, interaction_only = True)
features_polynomial = poly.fit_transform(X_train)

model_2 = LinearRegression(fit_intercept = True)
model_2.fit(features_polynomial, y_train)

y_pred_2 = model_2.predict( poly.transform(X) )

print ("Train Error:", MSE(y_train, y_pred_2[:train_size]) )
print ("Test Error:",  MSE(y_test, y_pred_2[train_size:]) )
print("R^2 = ", round(r2_score(y, y_pred_2), 5) )

Train Error: 0.5842003163147793
Test Error: 3.84992776900149
R^2 =  -0.31345


## TREE

In [30]:
def bias_var(X, y):
    t_size = int(len(X)*0.66)
    X_train = X[:t_size]
    y_train = y[:t_size]
    X_test = X[t_size:]
    y_test = y[t_size:]
    
    preds = []
    # The loop is iterated over a couple of equivalent parameters
    # to reduce the execution time, at the expense of reduction of results 
    for leave, samp in enumerate(range(2, 101)):
        dt = DecisionTreeClassifier(max_leaf_nodes = leave+2, min_samples_split = samp)
        model = dt.fit(X, y)
        preds += [ list(model.predict(X)) ] # every element is y_pred
        
    stats = []    
    for x in range(len(preds)):                           
        dt_variance = np.var(preds[x])
        dt_bias = MSE(y, preds[x]) - dt_variance  # bias^2 = MSE - variance    # (y - np.mean(preds[x]))**2
        acc = accuracy_score(y_true = y, y_pred = preds[x])
        stats += [ (dt_bias.mean(), dt_variance, round(acc, 6)) ]
        
    stats = [ list(x) for x in stats ]
    df = pd.DataFrame(stats, columns = ["bias", "variance", "accuracy"])
    df["bias_plus_var"] = df.bias + df.variance
    
    return df

In [31]:
stats_df = bias_var(X_tree, y_tree)

NameError: name 'X_tree' is not defined

In [ ]:
print(stats_df.loc[stats_df.bias_plus_var == min(stats_df.bias_plus_var)])
print()
print(stats_df.loc[stats_df.accuracy == max(stats_df.accuracy)])

To assess the parameter of the Tree Classifier, we performed a simulation to see how to set the ```max_leaf_nodes``` parameter in such a way that the the sum of bias$^2$ and Variance is minimized. This led to the value of ```39```, but we have seen a decrease in the accuracy of the model from 0.69 to 0.26.
<p>
*****The assement on ensemble method that is more suitable for this instance was made on the bias and variance value of the actual model, without the simulation implemented: since the variance is far low than the bias. <p>

```python
stats_df = opt_tree.bias_var(X_tree, y_tree) # long to be executed
print(stats_df.loc[stats_df.bias_plus_var == min(stats_df.bias_plus_var)])
print()
print(stats_df.loc[stats_df.accuracy == max(stats_df.accuracy)])

plt.style.use('fivethirtyeight')
fig, ax1 = plt.subplots(figsize = (9, 5))

ax1.set_xlabel('Model Complexity')
ax1.set_ylabel('Bias$^2$ + Variance', color = "#3DA5D9")
ax1.plot(range(len(stats_df)), stats_df.bias_plus_var, color = "#3DA5D9", linewidth = 1, alpha = 0.7)
ax1.plot(range(len(stats_df)), stats_df.variance, color = "#DB504A", linewidth = 1, alpha = 0.7)

ax1.tick_params(axis = 'y')

ax2 = ax1.twinx()
ax2.set_ylabel('Accuracy', color = "#EA7317")
ax2.plot(range(len(stats_df)), stats_df.accuracy, color = "#EA7317", linewidth = 1, alpha = 0.7)
ax2.tick_params(axis = 'y')
fig.tight_layout()
plt.savefig('Bias_variance_accuracy.png', dpi = 300)
```

In [53]:
X_tree = data.drop("appliance_wh", axis = 1).values
y_tree = data.appliance_wh
X_tree = X_tree.reshape( (len(X_tree), len(X_tree[0])))
y_tree = y_tree.values.reshape( len(y_tree), 1)

X_train_tree = X_tree[:train_size]
y_train_tree = y_tree[:train_size]
X_test_tree  = X_tree[train_size:]
y_test_tree  = y_tree[train_size:]

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

decisiontree = DecisionTreeClassifier(max_leaf_nodes = 39, min_samples_split = 39)
model_tree = decisiontree.fit(X_train_tree, y_train_tree)

acc = accuracy_score(y_true = y_tree, y_pred = model_tree.predict(X_tree))
print("Model Accuracy:", round(acc, 5))
print()
tree_pred = model_tree.predict(X_tree)

tree_variance = np.var(tree_pred)
tree_bias = MSE(y_tree, tree_pred) - tree_variance
tree_error = MSE(y_tree, tree_pred)

print("Bias^2:", round(tree_bias.mean(), 4), "--- Variance:", round(tree_variance, 4) )
print("MSE:", tree_error)
importance = dict(zip(data.drop("appliance_wh", axis = 1).columns, model_tree.feature_importances_))
importance = pd.DataFrame( sorted(importance.items(), key = operator.itemgetter(1))[::-1] ) # sort by value
importance = importance.rename(columns = {0: "col", 1:"val"})
#importance.head()

Model Accuracy: 0.24044

Bias^2: -342.4279 --- Variance: 343.0025
MSE: 0.574532494552825


In [50]:
def variance(l):
    var = [ ((l[x] - ( sum(l)/len(l) ))**2) for x in range(len(l)) ]
    return (sum(l)/len(l))

print(variance([2.75, 1.75, 1.25, 0.25, 0.5, 1.25, 3.5]))
print(statistics.variance([2.75, 1.75, 1.25, 0.25, 0.5, 1.25, 3.5]))

1.6071428571428572
1.3720238095238095


In [ ]:
#### BAGGING
from sklearn.ensemble import BaggingClassifier
bagged = BaggingClassifier(decisiontree)
bagged_2 = BaggingClassifier(bagged)
bagged_fit = bagged_2.fit(X_train_tree, y_train_tree)


print ("Accuracy:", accuracy_score(y_true = y_tree, y_pred = bagged_fit.predict(X_tree)) )

bagg_bias = (y_tree - np.mean(bagged_fit.predict(X_tree)))**2
bagg_variance = np.var(bagged_fit.predict(X_tree))
print("Bias^2:", round(bagg_bias.mean(), 4), "--- Variance:", round(bagg_variance, 4) )

In [ ]:
import datetime as dtime
#### BOOSTING
from sklearn.ensemble import AdaBoostClassifier
adaboost = AdaBoostClassifier(bagged_2)
print(dtime.datetime.now())
adaboost_2 = AdaBoostClassifier(adaboost)
print(dtime.datetime.now())
adaboost_fit = adaboost_2.fit(X_train_tree, y_train_tree)
print(dt.datetime.now())
print ("Accuracy:", accuracy_score(y_true = y_tree, y_pred = adaboost_fit.predict(X_tree)) )
ada_bias = (y_tree - np.mean(adaboost_fit.predict(X_tree)))**2
ada_variance = np.var(adaboost_fit.predict(X_tree))
print("Bias^2:", round(ada_bias.mean(), 4), "--- Variance:", round(ada_variance, 4) )


#### Stacking
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
#estimators = [ ("boos", adaboost), ("bagg", bagged) ]

#stack = StackingClassifier(estimators = estimators, final_estimator = LogisticRegression())
#stack_fit = stack.fit(X_train_tree, y_train_tree)

#print ("Accuracy:", accuracy_score(y_true = y_tree, y_pred = stack_fit.predict(X_tree)) )
#sta_bias = (y_tree - np.mean(stack_fit.predict(X_tree)))**2
#sta_variance = np.var(stack_fit.predict(X_tree))
#print("Bias^2:", round(sta_bias.mean(), 4), "--- Variance:", round(sta_variance, 4) )"""

In [ ]:
import graphviz
import pydotplus
import pydot
from IPython.display import Image
from graphviz import Graph
from sklearn import tree

dot_data = tree.export_graphviz(decisiontree, out_file = 'tree.dot',
                                filled = True, rounded = True, special_characters = True)

#(graph, ) = pydot.graph_from_dot_file('tree.dot')
#graph = graphviz.Source(dot_data)
#graph.write_png('tree.png')

# Draw graph
graph = pydot.graph_from_dot_data(dot_data)#.getvalue())
# Show graph
#Image(graph.create_png())
graph.write_png("tree.png")